## Creazione del dataset con i dati per il periodo di interesse

In [ ]:
import pandas as pd
import numpy as np
import datetime as DT
import requests
import json 
import warnings
import csv

indicators = [
    "mask",
    "worried_catch_covid",
    "belief_masking_effective",
    "received_news_local_health",
    "received_news_experts",
    "received_news_who",
    "received_news_govt_health",
    "received_news_politicians",
    "received_news_journalists",
    "received_news_friends",
    "trust_covid_info_local_health",
    "trust_covid_info_experts",
    "trust_covid_info_who",
    "trust_covid_info_govt_health",
    "trust_covid_info_politicians",
    "trust_covid_info_journalists",
    "trust_covid_info_friends"
]
countries = [
    "Argentina", "Australia", "Austria", "Belgium", "Canada", "Chile", "Colombia",
    "Czech Republic", "Denmark", "Ecuador", "Finland", "France", "Germany", "Greece",
    "Hungary", "Indonesia", "Italy", "Japan", "Malaysia", "Mexico", "Netherlands",
    "New Zealand", "Norway", "Peru", "Philippines", "Poland", "Portugal", "Romania",
    "Spain", "Sweden", "Switzerland", "Thailand", "Turkey", "United Kingdom"
]

df1 = pd.read_csv("csv/cd_long_v1.csv")   #BEH, BEL, AWA, TRU
df2 = pd.read_csv("csv/cs_long_v1.csv")
df = pd.concat([df1, df2], ignore_index=True)
df= df[df["country"].isin(countries)]
df= df[df["ind_name"].isin(indicators)]
#cambio formato tabella
df['survey_date'] = pd.to_datetime(df['survey_date'], errors='coerce')
df = df.pivot_table(
    index=["survey_date", "country"],   # chiavi
    columns="ind_name",                 
    values="pct_unw"                      
).reset_index()
df = df.rename(columns={col: f"pct_{col}" for col in indicators})
df = df.sort_values(by=["country", "survey_date"]).reset_index(drop=True)


In [ ]:
dg = pd.read_csv('csv/owid-covid-data.csv') #CASI, DEATHS, TEST
dg['date'] = pd.to_datetime(dg['date'])
dg= dg[dg['location'].isin(countries)]
dg=dg[["location", "date", "total_tests", "total_cases", "total_deaths", "new_cases_smoothed", "new_deaths_smoothed", "population"]]
dg = dg.rename(columns={
    'location': 'country',
    'date': 'survey_date'
})


In [ ]:
ds = pd.read_csv('csv/face-covering-policies-covid.csv') #FACE COVERING 
ds= ds[ds['Entity'].isin(countries)]
ds['Day'] = pd.to_datetime(ds['Day'])
ds=ds[["Entity", "Day", "Facial Coverings"]]

dr = pd.read_csv('csv/covid-containment-and-health-index.csv') #CHI
dr= dr[dr['Entity'].isin(countries)]
dr['Day'] = pd.to_datetime(dr['Day'])
dr=dr[["Entity", "Day", "Containment health index (average)"]]


dsr = pd.merge(
    ds,
    dr,
    on=['Entity', 'Day'],   # chiavi di unione
    how='outer'             # mantiene tutte le righe
)
dsr = dsr.rename(columns={
    'Entity': 'country',
    'Day': 'survey_date'
})


In [ ]:
dfg= pd.merge(
    df,
    dg,
    on=['country', 'survey_date'],
    how='outer'
)
dtot=pd.merge(
    dfg,
    dsr,
    on=['country', 'survey_date'],
    how='outer'
)
dtot= dtot[['survey_date', 'country', "pct_mask", "pct_belief_masking_effective","pct_worried_catch_covid", 
            "pct_received_news_local_health",
            "pct_received_news_experts",
            "pct_received_news_who",
            "pct_received_news_govt_health",
            "pct_received_news_politicians",
            "pct_received_news_journalists",
            "pct_received_news_friends",
            "pct_trust_covid_info_local_health",
            "pct_trust_covid_info_experts",
            "pct_trust_covid_info_who",
            "pct_trust_covid_info_govt_health",
            "pct_trust_covid_info_politicians",
            "pct_trust_covid_info_journalists",
            "pct_trust_covid_info_friends",
            "total_tests","total_cases","total_deaths","new_cases_smoothed","new_deaths_smoothed","Facial Coverings","Containment health index (average)", "population"]]
dtot.to_csv("csv/daticompletida2020.csv", index=False) #contiene tutti i dati dal 2020 in poi 
dtot

,survey_date,country,pct_mask,pct_belief_masking_effective,pct_worried_catch_covid,pct_received_news_local_health,pct_received_news_experts,pct_received_news_who,pct_received_news_govt_health,pct_received_news_politicians,...,pct_trust_covid_info_journalists,pct_trust_covid_info_friends,total_tests,total_cases,total_deaths,new_cases_smoothed,new_deaths_smoothed,Facial Coverings,Containment health index (average),population
0,2020-01-01,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,0.0,0.0,45510324.0
1,2020-01-02,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,99.0,NaN,NaN,NaN,NaN,0.0,0.0,45510324.0
2,2020-01-03,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,104.0,NaN,NaN,NaN,NaN,0.0,0.0,45510324.0
3,2020-01-04,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,180.0,NaN,NaN,NaN,NaN,0.0,0.0,45510324.0
4,2020-01-05,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,194.0,NaN,NaN,NaN,NaN,0.0,0.0,45510324.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52293,2024-04-03,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,24915646.0,232112.0,202.286,0.0,NaN,NaN,67508936.0
52294,2024-04-04,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,24915646.0,232112.0,202.286,0.0,NaN,NaN,67508936.0
52295,2024-04-05,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,24915646.0,232112.0,202.286,0.0,NaN,NaN,67508936.0
52296,2024-04-06,United Kingdom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,24915646.0,232112.0,202.286,0.0,NaN,NaN,67508936.0


In [ ]:
dtot['survey_date'] =pd.to_datetime(dtot['survey_date'])
dtot=dtot[dtot['survey_date'] >= '2020-04-23'] #prima mancano troppi dati
dtot=dtot[dtot['survey_date'] <= '2021-04-01']
dtot.to_csv("csv/dati.csv", index=False) #contiene tutti i dati dal 2020 in poi 
dtot

,survey_date,country,pct_mask,pct_belief_masking_effective,pct_worried_catch_covid,pct_received_news_local_health,pct_received_news_experts,pct_received_news_who,pct_received_news_govt_health,pct_received_news_politicians,...,pct_trust_covid_info_journalists,pct_trust_covid_info_friends,total_tests,total_cases,total_deaths,new_cases_smoothed,new_deaths_smoothed,Facial Coverings,Containment health index (average),population
113,2020-04-23,Argentina,0.787395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,51664.0,4066.0,279.0,153.714,10.571,3.0,86.31,45510324.0
114,2020-04-24,Argentina,0.791139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,54971.0,4066.0,279.0,153.714,10.571,3.0,86.31,45510324.0
115,2020-04-25,Argentina,0.768757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,57615.0,4066.0,279.0,153.714,10.571,3.0,86.31,45510324.0
116,2020-04-26,Argentina,0.789568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,59349.0,5464.0,411.0,199.714,18.857,3.0,85.12,45510324.0
117,2020-04-27,Argentina,0.798513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,61587.0,5464.0,411.0,199.714,18.857,3.0,79.17,45510324.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51191,2021-03-28,United Kingdom,0.819362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,115828717.0,4405263.0,152211.0,5426.714,93.429,3.0,69.40,67508936.0
51192,2021-03-29,United Kingdom,0.814665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,116842152.0,4405263.0,152211.0,5426.714,93.429,3.0,69.40,67508936.0
51193,2021-03-30,United Kingdom,0.798128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,117560747.0,4405263.0,152211.0,5426.714,93.429,3.0,69.40,67508936.0
51194,2021-03-31,United Kingdom,0.798090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,118859027.0,4405263.0,152211.0,5426.714,93.429,3.0,69.40,67508936.0
